*Step.1 導入相關套件*

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer ,AutoModelForQuestionAnswering , TrainingArguments , Trainer , DefaultDataCollator


*Step.2 載入數據*

In [ ]:
datasets = load_dataset("cmrc2018", cache_dir="data")

In [ ]:
datasets

In [ ]:
datasets["train"][0]

*Step.3 數據處理*

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
tokenizer

In [ ]:
sample_dataset = datasets["train"].select(range(10))

In [ ]:
tokenized_examples = tokenizer(text = sample_dataset["question"],
            text_pair = sample_dataset["context"],
            return_offsets_mapping = True,
            return_overflowing_tokens = True,
            stride = 128,
            max_length = 384, truncation="only_second", padding="max_length")


In [ ]:
tokenized_examples.keys()

In [ ]:
tokenized_examples["overflow_to_sample_mapping"], len(tokenized_examples["overflow_to_sample_mapping"])

In [ ]:
for sen in tokenizer.batch_decode(tokenized_examples["input_ids"][:3]):
    print(sen)

In [ ]:
print(tokenized_examples["offset_mapping"][:3])

In [ ]:
print(tokenized_examples["offset_mapping"][0], len(tokenized_examples["offset_mapping"][0]))

In [ ]:
sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

In [ ]:

for idx, _ in enumerate(sample_mapping):
    answer = sample_dataset["answers"][sample_mapping[idx]]
    start_char = answer["answer_start"][0]
    end_char = start_char + len(answer["text"][0])
    # 定位答案在token中的起始位置和结束位置
    # 拿到context的起始和結束位置，從左右兩側向答案靠近

    context_start = tokenized_examples.sequence_ids(idx).index(1)
    context_end = tokenized_examples.sequence_ids(idx).index(None, context_start) - 1

    offset = tokenized_examples.get("offset_mapping")[idx]
    example_ids = []

    # 判斷答案是否在context中
    if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
        start_token_pos = 0
        end_token_pos = 0
    else:
        token_id = context_start
        while token_id <= context_end and offset[token_id][0] < start_char:
            token_id += 1
        start_token_pos = token_id
        token_id = context_end
        while token_id >= context_start and offset[token_id][1] > end_char:
            token_id -=1
        end_token_pos = token_id
        example_ids.append([sample_mapping[idx]])

    print(answer, start_char, end_char, context_start, context_end, start_token_pos, end_token_pos)
    print("token answer decode:", tokenizer.decode(tokenized_examples["input_ids"][idx][start_token_pos: end_token_pos + 1]))


In [ ]:
def process_func(examples):
    tokenized_examples = tokenizer(text = examples["question"],
                text_pair = examples["context"],
                return_offsets_mapping = True,
                return_overflowing_tokens = True,
                stride = 128,
                max_length = 384, truncation = "only_second", padding = "max_length")
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    start_positions = []
    end_positions = []
    example_ids = []
    for idx, _ in enumerate(sample_mapping):
        answer = examples["answers"][sample_mapping[idx]]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])
        # 定位答案在token中的起始位置和结束位置
        # 拿到context的起始和結束位置，從左右兩側向答案靠近
        context_start = tokenized_examples.sequence_ids(idx).index(1)
        context_end = tokenized_examples.sequence_ids(idx).index(None, context_start) - 1
        offset = tokenized_examples.get("offset_mapping")[idx]
        # 判斷答案是否在context中
        if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
            start_token_pos = 0
            end_token_pos = 0
        else:
            token_id = context_start
            while token_id <= context_end and offset[token_id][0] < start_char:
                token_id += 1
            start_token_pos = token_id
            token_id = context_end
            while token_id >= context_start and offset[token_id][1] > end_char:
                token_id -=1
            end_token_pos = token_id
        start_positions.append(start_token_pos)
        end_positions.append(end_token_pos)
        example_ids.append(examples["id"][sample_mapping[idx]])
        tokenized_examples["offset_mapping"][idx] = [
            (o if tokenized_examples.sequence_ids(idx)[k] == 1 else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][idx])
        ]


    tokenized_examples["example_ids"] = example_ids
    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    return tokenized_examples

In [ ]:
tokenied_datasets = datasets.map(process_func, batched=True, remove_columns=datasets["train"].column_names)
tokenied_datasets

In [ ]:
print(tokenied_datasets["train"]["offset_mapping"][1])

In [ ]:
tokenied_datasets["train"]["example_ids"][:10]

In [ ]:
import collections
# example轉換feature
example_to_feature = collections.defaultdict(list)
for idx, example_id in enumerate(tokenied_datasets["train"]["example_ids"][:10]):
    example_to_feature[example_id].append(idx)
example_to_feature

*Step.4 模型輸出*

In [ ]:
import numpy as np
import collections

def get_result(start_logits, end_logits, exmaples, features):

    predictions = {}
    references = {}

    # example轉換feature
    example_to_feature = collections.defaultdict(list)
    for idx, example_id in enumerate(features["example_ids"]):
        example_to_feature[example_id].append(idx)

    # 最優解
    n_best = 20
    # 最大答案長度
    max_answer_length = 30

    for example in exmaples:
        example_id = example["id"]
        context = example["context"]
        answers = []
        for feature_idx in example_to_feature[example_id]:
            start_logit = start_logits[feature_idx]
            end_logit = end_logits[feature_idx]
            offset = features[feature_idx]["offset_mapping"]
            start_indexes = np.argsort(start_logit)[::-1][:n_best].tolist()
            end_indexes = np.argsort(end_logit)[::-1][:n_best].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if offset[start_index] is None or offset[end_index] is None:
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    answers.append({
                        "text": context[offset[start_index][0]: offset[end_index][1]],
                        "score": start_logit[start_index] + end_logit[end_index]
                    })
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["score"])
            predictions[example_id] = best_answer["text"]
        else:
            predictions[example_id] = ""
        references[example_id] = example["answers"]["text"]

    return predictions, references

*Step.5 評估函數*

In [ ]:
from cmrc_eval import evaluate_cmrc

def metirc(pred):
    start_logits, end_logits = pred[0]
    if start_logits.shape[0] == len(tokenied_datasets["validation"]):
        p, r = get_result(start_logits, end_logits, datasets["validation"], tokenied_datasets["validation"])
    else:
        p, r = get_result(start_logits, end_logits, datasets["test"], tokenied_datasets["test"])
    return evaluate_cmrc(p, r)

*Step.6 建立模型*

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained("hfl/chinese-macbert-base")

*Step.7 訓練參數*

In [ ]:
args = TrainingArguments(
    output_dir = "models_for_qa",
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    eval_strategy = "steps",
    eval_steps = 200,
    save_strategy = "epoch",
    logging_steps = 50,
    num_train_epochs = 1
)

*Step.8 訓練器*

In [ ]:
trainer = Trainer(
    model = model,
    args = args,
    tokenizer = tokenizer,
    train_dataset = tokenied_datasets["train"],
    eval_dataset = tokenied_datasets["validation"],
    data_collator = DefaultDataCollator()
)

*Step.9 模型訓練*

In [ ]:
trainer.train()

*Step.8 模型預測*

In [ ]:
from transformers import pipeline

pipe = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)

In [ ]:
pipe(question="小明在哪裡上班？", context="小明在北京上班。")